In [1]:
import numpy as np
import pandas as pd
import os
import sys
import glob
import time 
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import current_date
from fbprophet import Prophet
from reduce_mem_usage import reduce_mem_usage
from continuous_data import round_target_column
from surface_preprocessing import create_daily_test, surface_preprocess

##### Metric Evaluation function

In [2]:
import os
os.chdir('../')

In [3]:
#### process the new data and append the old data 

new_data = surface_preprocess()

In [3]:
data = pd.read_csv('input_files/surface_data.csv')
data = reduce_mem_usage(data)


#convert the sales date into datetime format 

data['Sales Date'] = pd.to_datetime(data['Sales Date'])
data = data.drop_duplicates()


#data = (data.set_index("Sales Date").groupby(['Reseller City','Super Division', 'Business Unit', pd.Grouper(freq='W')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())
data['black_week'] = np.where((data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.day > 23), 1, 0)

max_date = data['Sales Date'].max()

Mem. usage decreased to 168.69 Mb (21.9% reduction)


In [4]:
max_date

Timestamp('2022-02-26 00:00:00')

In [5]:
closed_stores = ['Saturn Connect Trier', 'Media Markt Heilbronn 2','Saturn Schweinfurt Schrammstraße', 'Saturn Connect Köln',
                    'Saturn Stuttgart-Hbf', 'Saturn-Berlin Clayallee','Saturn Mönchengladbach - Stresemannstraße', 'Saturn Lübeck',
                    'Saturn München Theresienhöhe', 'Saturn Berlin-Alt-Treptow','Media Markt Turnstraße', 'Saturn Wiesbaden Bahnhofsplatz',
                    'Media Markt Ellwangen - DAS', 'Media Markt GmbH Nürtingen','Media Markt Meppen', 'Media Markt Schleswig', 
                    'Media-Saturn IT Services GmbH', 'Meida Markt Waiblingen', 'Saturn Bergisch Gladbach','Saturn Wesel', 'Saturn Hagen', 
                    'Media Markt Bad Cannstatt', 'Saturn Heidelberg', 'Saturn Hildesheim', 'Saturn Münster am York-Ring', 'Media Markt Köln-Chorweiler',
                    'Saturn Dessau', 'Saturn Essen-Steele','Saturn Euskirchen', 'Saturn Göttingen', 'Saturn Hennef', 'Saturn Herford',
                    'Saturn Düsseldorf','Saturn Itzehoe','Saturn Siegburg','Saturn Weiterstadt', 'Saturn Bremerhaven - BSS', 'Saturn Gelsenkirchen Buer']
data = data.loc[~data['Store_names'].isin(closed_stores)].reset_index(drop=True)

In [6]:
data.nunique()

Sales Date               1735
Store_names               393
Reseller City             257
Super Division              1
Business Unit               5
Rslr Sales Quantity       131
Rslr Sales Amount       15098
Reseller Postal Code      378
black_week                  2
dtype: int64

In [7]:
data.shape


(3384225, 9)

In [8]:
data.isna().sum()

Sales Date              0
Store_names             0
Reseller City           0
Super Division          0
Business Unit           0
Rslr Sales Quantity     0
Rslr Sales Amount       0
Reseller Postal Code    0
black_week              0
dtype: int64

In [9]:
import calendar as cal
from datetime import *
from dateutil.relativedelta import *
import holidays

def black_week(year):
    black_friday = (datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4)) - timedelta(days=4)), datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4))+ timedelta(days=1)))
    cyber_monday = datetime.date(datetime(year, 11, 1) + relativedelta(weekday=FR(+4))+ timedelta(days=3))

    black_friday = pd.to_datetime(black_friday)
    black_friday =pd.date_range(black_friday[0], black_friday[1], freq ='d')

    cyber_monday = pd.to_datetime(cyber_monday)
    #cyber_monday = list(cyber_monday)


    return black_friday, cyber_monday

In [39]:
last_month_days =  list(cal.monthrange(max_date.year, max_date.month))

remain_days = last_month_days[1] - max_date.day



In [40]:
remain_days

2

In [41]:
next_month_days = list(cal.monthrange(max_date.year, max_date.month+1))

future_days = remain_days+next_month_days[1]

In [44]:
max_date+pd.to_timedelta(future_days,unit='d')

Timestamp('2022-03-31 00:00:00')

In [42]:
future_days

33

In [10]:
# ## create the test_dataframe 
test_data = create_daily_test(data, max_date)
test_data['Sales Date'] = pd.to_datetime(test_data['Sales Date'])
test_data['black_week'] = np.where((test_data['Sales Date'].dt.month==11) & (test_data['Sales Date'].dt.day > 23), 1, 0)

In [11]:
test_data['Sales Date'].unique()

array(['2022-02-27T00:00:00.000000000', '2022-02-28T00:00:00.000000000',
       '2022-03-01T00:00:00.000000000', '2022-03-02T00:00:00.000000000',
       '2022-03-03T00:00:00.000000000', '2022-03-04T00:00:00.000000000',
       '2022-03-05T00:00:00.000000000', '2022-03-06T00:00:00.000000000',
       '2022-03-07T00:00:00.000000000', '2022-03-08T00:00:00.000000000',
       '2022-03-09T00:00:00.000000000', '2022-03-10T00:00:00.000000000',
       '2022-03-11T00:00:00.000000000', '2022-03-12T00:00:00.000000000',
       '2022-03-13T00:00:00.000000000', '2022-03-14T00:00:00.000000000',
       '2022-03-15T00:00:00.000000000', '2022-03-16T00:00:00.000000000',
       '2022-03-17T00:00:00.000000000', '2022-03-18T00:00:00.000000000',
       '2022-03-19T00:00:00.000000000', '2022-03-20T00:00:00.000000000',
       '2022-03-21T00:00:00.000000000', '2022-03-22T00:00:00.000000000',
       '2022-03-23T00:00:00.000000000', '2022-03-24T00:00:00.000000000',
       '2022-03-25T00:00:00.000000000', '2022-03-26

In [12]:
data = pd.concat([data, test_data])

In [13]:
data = (data.set_index("Sales Date").groupby(['Store_names','Reseller City','Super Division','Business Unit', pd.Grouper(freq='M')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())

In [14]:
### get the promotional data

promos = pd.read_csv('input_files/promotion_data/Promos_data.csv')

promos = promos.rename(columns = {'Sales_Date' : 'Sales Date',
                                    'Business unit': 'Business Unit'})

promos['Sales Date'] = pd.to_datetime(promos['Sales Date'])

### convert the promos to weekly data
promos = (promos.set_index("Sales Date").groupby(['Business Unit',pd.Grouper(freq='M')])["Discount_amount"].sum().reset_index())

promos = promos.drop('Discount_amount', 1)
promos['promos'] = 1
### merge the promos with surface data 

data = pd.merge(data, promos, on=['Sales Date', 'Business Unit'], how='left')
data['promos'] = data['promos'].fillna(0)

In [15]:
data.head()

,Store_names,Reseller City,Super Division,Business Unit,Sales Date,Rslr Sales Quantity,Rslr Sales Amount,promos
0,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-01-31,1,1351,0.0
1,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-02-28,0,0,0.0
2,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-03-31,0,0,0.0
3,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-04-30,0,0,0.0
4,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-05-31,0,0,0.0


In [16]:
data.shape

(123803, 8)

In [17]:
data['black_week'] = np.where(data['Sales Date'].dt.month==11, 1, 0)

In [18]:
data['black_week'] = np.where((data['Business Unit']=='Surface Pro') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year== 2017 |2018 |2019), 1, 0)
data['black_week'] = np.where((data['Business Unit']=='Surface Pro') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2020), 2, data['black_week'])
data['black_week'] = np.where((data['Business Unit']=='Surface Go') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2019 |2021), 1, data['black_week'])
data['black_week'] = np.where((data['Business Unit']=='Surface Book') & (data['Sales Date'].dt.month==11) & (data['Sales Date'].dt.year==2018 |2019), 1, data['black_week'])


In [19]:
data['christmas'] = np.where((data['Business Unit']=='Surface Pro') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2017 |2018 |2019 |2020), 1, 0)
data['christmas'] = np.where((data['Business Unit']=='Surface Go') & (data['Sales Date'].dt.month==12) & (data['Sales Date'].dt.year==2018 |2019 |2020 |2021), 1, data['christmas'])

In [20]:
data

,Store_names,Reseller City,Super Division,Business Unit,Sales Date,Rslr Sales Quantity,Rslr Sales Amount,promos,black_week,christmas
0,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-01-31,1,1351,0.0,0,0
1,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-02-28,0,0,0.0,0,0
2,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-03-31,0,0,0.0,0,0
3,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-04-30,0,0,0.0,0,0
4,Saturn Köln-Weiden,Köln,EDG Managed - Surface Devices,Surface Book,2017-05-31,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
123798,Saturn Zwickau,Zwickau,EDG Managed - Surface Devices,Surface Pro,2021-11-30,6,4668,0.0,0,0
123799,Saturn Zwickau,Zwickau,EDG Managed - Surface Devices,Surface Pro,2021-12-31,1,921,0.0,0,0
123800,Saturn Zwickau,Zwickau,EDG Managed - Surface Devices,Surface Pro,2022-01-31,1,1406,0.0,0,0
123801,Saturn Zwickau,Zwickau,EDG Managed - Surface Devices,Surface Pro,2022-02-28,1,925,0.0,0,0


In [21]:
data.isnull().values.any()

False

In [22]:
import plotly.graph_objs as go
def plot_fig(forecast):
    
    forecast = (forecast.set_index("Sales Date").groupby(['Super Division','Business Unit', pd.Grouper(freq='M')])["Rslr Sales Quantity", "Rslr Sales Amount"].sum().astype(int).reset_index())
    forecast1 = forecast.loc[forecast['Business Unit']=='Surface Book']
    forecast2 = forecast.loc[forecast['Business Unit']=='Surface Go']
    forecast3 = forecast.loc[forecast['Business Unit']=='Surface Laptop']
    forecast4 = forecast.loc[forecast['Business Unit']=='Surface Laptop Go']
    forecast5 = forecast.loc[forecast['Business Unit']=='Surface Pro']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast1['Sales Date'], y=forecast1['Rslr Sales Quantity'], name='Surface Book',))
    fig.add_trace(go.Scatter(x=forecast2['Sales Date'], y=forecast2['Rslr Sales Quantity'], name='Surface Go',))
    #fig.add_trace(go.Scatter(x=forecast3['Sales Date'], y=forecast3['Rslr Sales Quantity'], name='Surface Laptop',))
    #fig.add_trace(go.Scatter(x=forecast4['Sales Date'], y=forecast4['Rslr Sales Quantity'], name='Surface Laptop Go',))
    fig.add_trace(go.Scatter(x=forecast5['Sales Date'], y=forecast5['Rslr Sales Quantity'], name='Surface Pro',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['promos'], name='promos',))
    fig.show()

In [23]:
plot_fig(data)

In [24]:

# spark = SparkSession.builder\
#         .appName('Surface_items_forecast') \
#         .master('local[*]') \
#         .config('spark.sql.execution.arrow.pyspark.enabled', True) \
#         .config('spark.sql.execution.arrow.enabled', True) \
#         .config('spark.sql.session.timeZone', 'UTC') \
#         .config('spark.executor.memory','16g') \
#         .config('spark.driver.memory','16g') \
#         .config("spark.driver.maxResultSize", "4g") \
#         .getOrCreate()
         #.config('spark.sql.repl.eagerEval.enabled', True) \
         #.config('spark.ui.showConsoleProgress', True) \
import time
spark = SparkSession.builder\
        .appName('surface_monthly_forecast') \
        .master('local[*]') \
        .config('spark.sql.execution.arrow.pyspark.enabled', True) \
        .config('spark.sql.execution.arrow.enabled', True) \
        .getOrCreate()

start_time = time.time()
sdf = spark.createDataFrame(data)
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))
sdf = sdf.withColumnRenamed('Sales Date', 'ds')\
        .withColumnRenamed('Super Division', 'Super_Division')\
        .withColumnRenamed('Rslr Sales Quantity', 'Rslr_Sales_Qunatity')\
        .withColumnRenamed('Reseller City', 'Reseller_City')\
        .withColumnRenamed('Business Unit', 'Business_Unit')

sdf.printSchema()

sdf.createOrReplaceTempView('sales')

0.03 min: Lags
root
 |-- Store_names: string (nullable = true)
 |-- Reseller_City: string (nullable = true)
 |-- Super_Division: string (nullable = true)
 |-- Business_Unit: string (nullable = true)
 |-- ds: timestamp (nullable = true)
 |-- Rslr_Sales_Qunatity: integer (nullable = true)
 |-- Rslr Sales Amount: integer (nullable = true)
 |-- promos: double (nullable = true)
 |-- black_week: integer (nullable = true)
 |-- christmas: integer (nullable = true)



In [25]:


#spark.sql("select Store_names, Reseller_City,  Business_Unit, count(*) from sales group by Store_names, Reseller_City, Business_Unit order by Reseller_City, Business_Unit").show()

sql = 'SELECT Store_names, Reseller_City, Super_Division, Business_Unit, black_week, promos, christmas, ds, sum(Rslr_Sales_Qunatity) as y FROM sales GROUP BY Store_names, Reseller_City, Super_Division, Business_Unit, black_week, promos, christmas, ds ORDER BY Store_names, Reseller_City,  Super_Division,  Business_Unit, ds'

sdf.explain()


== Physical Plan ==
*(1) Project [Store_names#0, Reseller City#1 AS Reseller_City#53, Super Division#2 AS Super_Division#31, Business Unit#3 AS Business_Unit#64, Sales Date#4 AS ds#20, Rslr Sales Quantity#5 AS Rslr_Sales_Qunatity#42, Rslr Sales Amount#6, promos#7, black_week#8, christmas#9]
+- *(1) Scan ExistingRDD arrow[Store_names#0,Reseller City#1,Super Division#2,Business Unit#3,Sales Date#4,Rslr Sales Quantity#5,Rslr Sales Amount#6,promos#7,black_week#8,christmas#9]




In [26]:
sdf.rdd.getNumPartitions()

8

In [27]:
spark.sql("select Store_names, Reseller_City,  Business_Unit, count(*) from sales group by Store_names, Reseller_City, Business_Unit order by Reseller_City, Business_Unit").show()

+--------------------+-------------+-----------------+--------+
|         Store_names|Reseller_City|    Business_Unit|count(1)|
+--------------------+-------------+-----------------+--------+
|  Media Markt Aachen|       Aachen|     Surface Book|      63|
|       Saturn Aachen|       Aachen|     Surface Book|      63|
|  Media Markt Aachen|       Aachen|       Surface Go|      63|
|       Saturn Aachen|       Aachen|       Surface Go|      63|
|  Media Markt Aachen|       Aachen|   Surface Laptop|      63|
|       Saturn Aachen|       Aachen|   Surface Laptop|      63|
|  Media Markt Aachen|       Aachen|Surface Laptop Go|      63|
|       Saturn Aachen|       Aachen|Surface Laptop Go|      63|
|  Media Markt Aachen|       Aachen|      Surface Pro|      63|
|       Saturn Aachen|       Aachen|      Surface Pro|      63|
|   Media Markt Aalen|        Aalen|     Surface Book|      63|
|   Media Markt Aalen|        Aalen|       Surface Go|      63|
|   Media Markt Aalen|        Aalen|   S

In [28]:
### make partitions of data based on number of cores in the local system
#sdf.rdd.getNumPartitions()

spark.sql(sql).show()

store_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['Store_names','Business_Unit'])).cache()
store_part.explain()

+-------------------+-------------+--------------------+-------------+----------+------+---------+-------------------+---+
|        Store_names|Reseller_City|      Super_Division|Business_Unit|black_week|promos|christmas|                 ds|  y|
+-------------------+-------------+--------------------+-------------+----------+------+---------+-------------------+---+
| Saturn Köln-Weiden|         Köln|EDG Managed - Sur...| Surface Book|         0|   0.0|        0|2017-01-31 00:00:00|  1|
| Saturn Köln-Weiden|         Köln|EDG Managed - Sur...| Surface Book|         0|   0.0|        0|2017-02-28 00:00:00|  0|
| Saturn Köln-Weiden|         Köln|EDG Managed - Sur...| Surface Book|         0|   0.0|        0|2017-03-31 00:00:00|  0|
| Saturn Köln-Weiden|         Köln|EDG Managed - Sur...| Surface Book|         0|   0.0|        0|2017-04-30 00:00:00|  0|
| Saturn Köln-Weiden|         Köln|EDG Managed - Sur...| Surface Book|         0|   0.0|        0|2017-05-31 00:00:00|  0|
| Saturn Köln-We

In [29]:
### make the resultant schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('Store_names',StringType()),
    StructField('Reseller_City',StringType()),
    StructField('Super_Division',StringType()),
    StructField('Business_Unit',StringType()),
    StructField('black_week',DoubleType()),
    StructField('christmas',DoubleType()),
    StructField('y',DoubleType()),
    StructField('yhat',DoubleType()),
    StructField('yhat_upper',DoubleType()),
    StructField('yhat_lower',DoubleType())
])

In [30]:
### create the holiday dataframe 

lockdown1 = pd.date_range('2020-03-22', '2020-05-03', freq ='m').to_list()
lockdown2 = pd.date_range('2020-12-13', '2021-03-07', freq ='m').to_list()
lockdown = lockdown1+lockdown2


lock_down = pd.DataFrame({
    'holiday': 'lock_down',
    'ds' : pd.to_datetime(lockdown)})


In [31]:
##### city-wise prophet function 
@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_sales( store_pd):
    
    model = Prophet(interval_width=0.95, holidays = lock_down)
    model.add_country_holidays(country_name='DE')
    model.add_regressor('black_week')
    model.add_regressor('promos')
    model.add_regressor('christmas')

    black_week = dict(zip(store_pd['ds'], store_pd['black_week']))
    christmas = dict(zip(store_pd['ds'], store_pd['christmas']))
    promos_dates = store_pd.loc[store_pd['promos'] == 1]['ds'].unique()
    train = store_pd[store_pd['ds']<='2022-02-28']
    future_pd = store_pd[store_pd['ds']>'2022-02-28'].set_index('ds')

    def conditions(data):
        if data['ds'] in promos_dates:
            return 1
        else:
            return 0

    train['date_index'] = train['ds']
    train['date_index'] = pd.to_datetime(train['date_index'])
    train = train.set_index('date_index')

    model.fit(train[['ds', 'y', 'black_week', 'promos', 'christmas']])
    future = model.make_future_dataframe(periods=1, freq='m')
    future['promos'] = future.apply(conditions, axis=1)
    future['black_week'] = future['ds'].map(black_week)
    future['christmas'] = future['ds'].map(christmas)

    forecast_pd = model.predict(future[['ds', 'black_week', 'promos', 'christmas']])
    f_pd = forecast_pd[['ds', 'yhat', 'yhat_upper', 'yhat_lower', 'black_week', 'christmas']].set_index('ds')

    #forecast_pd = model.predict(future_pd[['ds', 'black_week', 'EOL_Promos']])  

    #store_pd = store_pd.filter(store_pd['ds']<'2021-10-01 00:00:00')
    st_pd = store_pd[[ 'ds', 'Store_names', 'Reseller_City', 'Super_Division', 'Business_Unit', 'y']].set_index('ds')
    #st_pd = pd.concat([st_pd1, st_pd2])

    results_pd = f_pd.join( st_pd, how='left' )
    results_pd.reset_index(level=0, inplace=True)

    #results_pd[['Reseller_City','Business_Unit']] = future_pd[['Reseller_City','Business_Unit']].iloc[0]
    return results_pd[['ds', 'Store_names', 'Reseller_City', 'Super_Division', 'Business_Unit', 'black_week', 'christmas', 'y', 'yhat', 'yhat_upper', 'yhat_lower']]


In [32]:
#### run the spark by city-wise
import time
results = (
    store_part
    .groupBy(['Store_names','Business_Unit'])
    .apply(forecast_sales)
    .withColumn('training_date', current_date() )
    )

### cache the results 
start_time = time.time()
results.cache()
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

results.explain()
results = results.coalesce(1)



0.00 min: Lags
== Physical Plan ==
InMemoryTableScan [ds#357, Store_names#358, Reseller_City#359, Super_Division#360, Business_Unit#361, black_week#362, christmas#363, y#364, yhat#365, yhat_upper#366, yhat_lower#367, training_date#379]
   +- InMemoryRelation [ds#357, Store_names#358, Reseller_City#359, Super_Division#360, Business_Unit#361, black_week#362, christmas#363, y#364, yhat#365, yhat_upper#366, yhat_lower#367, training_date#379], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) Project [ds#357, Store_names#358, Reseller_City#359, Super_Division#360, Business_Unit#361, black_week#362, christmas#363, y#364, yhat#365, yhat_upper#366, yhat_lower#367, 2022-03-03 AS training_date#379]
            +- FlatMapGroupsInPandas [Store_names#0, Business_Unit#64], forecast_sales(Store_names#0, Reseller_City#53, Super_Division#31, Business_Unit#64, black_week#8, promos#7, christmas#9, ds#20, y#151L), [ds#357, Store_names#358, Reseller_City#359, Super_Division#360, Busines

In [33]:
### convert the result from sparkdataframe to panadas datafrme 
start_time = time.time()
final_df = results.toPandas()

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

255.52 min: Lags


In [34]:
final_df['yhat'] = np.where(final_df['yhat']<0, 0, final_df['yhat'])
final_df['yhat_upper'] = np.where(final_df['yhat_upper']<0, 0, final_df['yhat_upper'])
final_df['yhat_lower'] = np.where(final_df['yhat_lower']<0, 0, final_df['yhat_lower'])

In [35]:
test_results = final_df.loc[final_df['ds']>='2022-02-28']

In [48]:
test_results

,ds,Store_names,Reseller_City,Super_Division,Business_Unit,black_week,christmas,y,yhat,yhat_upper,yhat_lower,training_date
61,2022-02-28,MM Stade,Stade,EDG Managed - Surface Devices,Surface Pro,-0.152781,0.0,2.0,2.125792,4.466513,0.000000,2022-03-03
62,2022-03-31,MM Stade,Stade,EDG Managed - Surface Devices,Surface Pro,-0.152781,0.0,0.0,2.384950,4.710005,0.206817,2022-03-03
124,2022-02-28,Media Markt Aachen,Aachen,EDG Managed - Surface Devices,Surface Book,0.000000,0.0,1.0,0.353104,2.526093,0.000000,2022-03-03
125,2022-03-31,Media Markt Aachen,Aachen,EDG Managed - Surface Devices,Surface Book,0.000000,0.0,0.0,0.665352,2.700862,0.000000,2022-03-03
187,2022-02-28,Media Markt Albstadt,Albstadt,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,1.0,0.000000,0.936872,0.000000,2022-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...
123676,2022-03-31,Saturn Witten,Witten,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,0.000000,1.056126,0.000000,2022-03-03
123738,2022-02-28,Saturn Wolfsburg City Galerie,Wolfsburg,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,0.363960,1.239205,0.000000,2022-03-03
123739,2022-03-31,Saturn Wolfsburg City Galerie,Wolfsburg,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,1.201395,2.171464,0.251437,2022-03-03
123801,2022-02-28,Saturn Wuppertal-Elberfeld,Wuppertal,EDG Managed - Surface Devices,Surface Pro,-0.542467,0.0,8.0,5.587787,13.324839,0.000000,2022-03-03


In [37]:
from sklearn.metrics import mean_absolute_error
rmse_pred = mean_absolute_error(test_results['y'], test_results['yhat'])
    
print("Root Mean Absolute Error_store:" , np.sqrt(rmse_pred))

Root Mean Absolute Error_store: 1.3822127507965942


In [38]:
normalize_rmse = np.sqrt(rmse_pred)/(test_results['y'].max()-test_results['y'].min())
print("Normalize RMSE:" , normalize_rmse)

Normalize RMSE: 0.006170592637484796


In [39]:
stores=final_df['Store_names'].unique()
stores = final_df['Store_names'].unique()
product=final_df['Business_Unit'].unique()

In [40]:
import plotly.graph_objs as go
def plot_fig(df, store, item):
    fig = go.Figure()
    # Create and style traces
    forecast = df.loc[df['Store_names'] == store]
    print(store, item)
    forecast = forecast.loc[forecast['Business_Unit'] == item]
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['y'], name='Actual',))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], name='lowelimit',))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], name='upperlimit',))
    #fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['promos'], name='promos',))
    fig.show()

In [51]:
plot_fig(final_df, stores[100], product[0])

Media Markt Rostock-Sievershagen Surface Pro


In [49]:
final_df = final_df.drop_duplicates()
final_df.to_csv(r'output_files/surface_store_historic.csv', index=False)

predicted_data = final_df.loc[final_df['ds']> '2022-02-28']

predicted_data = round_target_column(predicted_data, 'yhat')
predicted_data = round_target_column(predicted_data, 'yhat_lower')
predicted_data = round_target_column(predicted_data, 'yhat_upper')

predicted_data.to_csv(r'output_files/surface_store_monthly.csv', index=False)

In [50]:
predicted_data

,ds,Store_names,Reseller_City,Super_Division,Business_Unit,black_week,christmas,y,yhat,yhat_upper,yhat_lower,training_date
62,2022-03-31,MM Stade,Stade,EDG Managed - Surface Devices,Surface Pro,-0.152781,0.0,0.0,2,4,0,2022-03-03
125,2022-03-31,Media Markt Aachen,Aachen,EDG Managed - Surface Devices,Surface Book,0.000000,0.0,0.0,0,2,0,2022-03-03
188,2022-03-31,Media Markt Albstadt,Albstadt,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,0,1,0,2022-03-03
251,2022-03-31,Media Markt Alzey,Alzey,EDG Managed - Surface Devices,Surface Pro,-0.213128,0.0,0.0,4,7,0,2022-03-03
314,2022-03-31,Media Markt Amberg,Amberg,EDG Managed - Surface Devices,Surface Laptop Go,0.000000,0.0,0.0,0,0,0,2022-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...
123550,2022-03-31,Saturn Webshop,Ingolstadt,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,41,60,22,2022-03-03
123613,2022-03-31,Saturn Wiesbaden,Wiesbaden,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,0,1,0,2022-03-03
123676,2022-03-31,Saturn Witten,Witten,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,0,1,0,2022-03-03
123739,2022-03-31,Saturn Wolfsburg City Galerie,Wolfsburg,EDG Managed - Surface Devices,Surface Laptop,0.000000,0.0,0.0,1,2,0,2022-03-03


In [54]:
(final_df.loc[final_df['ds']>= '2022-01-31'].set_index("ds").groupby(['Business_Unit', pd.Grouper(freq='M')])["y", 'yhat'].sum().astype(int).reset_index())

,Business_Unit,ds,y,yhat
0,Surface Book,2022-01-31,129,365
1,Surface Book,2022-02-28,220,231
2,Surface Book,2022-03-31,0,453
3,Surface Go,2022-01-31,795,1390
4,Surface Go,2022-02-28,727,1108
5,Surface Go,2022-03-31,0,1490
6,Surface Laptop,2022-01-31,379,365
7,Surface Laptop,2022-02-28,291,338
8,Surface Laptop,2022-03-31,0,506
9,Surface Laptop Go,2022-01-31,341,390
